<center>
  <h1>Assignment 1: Developing Flappy Bird using pygame</h1>
</center>

![pygame logo](https://www.pygame.org/docs/_static/pygame_logo.png)


This Python Notebook provides a code template where you'll need to complete logical sections of the code. After succesfully running the game, please submit the link to your colab notebook or drive over [here](https://forms.gle/9udc1qcoWDsBodF88).

## Import necessary libraries

In [3]:
import pygame
import random

## Define global constants

In [4]:
# Game Variables 
# NOTE : Press any key to restart 
WIDTH, HEIGHT = 1024, 576
GRAVITY = 0.25
BIRD_JUMP = 7
PIPE_GAP = 200
PIPE_VELOCITY = 3
BIRD_WIDTH, BIRD_HEIGHT = 30, 30
PIPE_WIDTH, PIPE_HEIGHT = 50, 800
BACKGROUND_COLOR = (0, 102, 204)


## Load assets

In [5]:
# Loading Images
pipe_image = pygame.image.load('pipe.png')
pipe_image = pygame.transform.scale(pipe_image,(PIPE_WIDTH, PIPE_HEIGHT))

## Initialize Pygame and Set Up the Display

In [6]:
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
clock = pygame.time.Clock()
font = pygame.font.SysFont('dejavuserif',60)

## Making Bird and Pipe Classes

In [7]:
# Bird class
class Bird(pygame.sprite.Sprite):
    def __init__(self):
        pygame.sprite.Sprite.__init__(self)
        self.state = [] #To store different flapping positions of bird
        self.index = 0
        for num in range(1,5) : # there should be bird1.png,bird2.png,bird3.png and bird4.png in the directory
            image = pygame.image.load(f'bird{num}.png').convert_alpha()
            image = pygame.transform.scale(image, (BIRD_HEIGHT,BIRD_WIDTH))
            self.state.append(image)
        self.image = self.state[self.index]  
        self.rect = self.image.get_rect()
        self.rect.center = (300,250)
        self.vel = -10
        self.counter = 0

    def jump(self):
        if pygame.mouse.get_pressed()[0] :
            self.vel = -1* BIRD_JUMP
        if self.vel > 10 :
            self.vel = 10

    def update(self):
        self.rect.y += int(self.vel)
        self.vel += GRAVITY
        self.counter += 1
        if self.counter >= 20 :
            self.counter = 0 
            self.index += 1
        if self.index >= 4 :
            self.index = 0 
        self.image = self.state[self.index]

    def draw(self):
        bird_group.draw(screen)

# Pipe class
class Pipe(pygame.sprite.Sprite):
    def __init__(self,x,y,inverted):
        pygame.sprite.Sprite.__init__(self)
        image = pygame.image.load('pipe.png')
        self.image = pygame.transform.scale(image,(PIPE_WIDTH,PIPE_HEIGHT))
        if inverted == True :
            self.image = pygame.transform.flip(self.image, False, True)
            self.rect = self.image.get_rect()
            self.rect.midbottom = (x,y - PIPE_GAP)
        else:
            self.rect = self.image.get_rect()
            self.rect.midtop = (x,y)

    def update(self) :
        self.rect.x -= PIPE_VELOCITY
        if self.rect.x <= -20 :
            self.kill()

# setting the bird and pipes instances
last_pipe = 0          
bird_group = pygame.sprite.Group()
bird = Bird()
bird_group.add(bird)          
pipe_group = pygame.sprite.Group()
running = True
Active = True
pass_pipe = False
score = 0


## Collision detection function

In [8]:
# Checking collisions
def check_collision(group_1,group_2) :
    if pygame.sprite.groupcollide(group_1,group_2,False,False) :  
        return False
    else :
        return True

## Main Game Loop

In [9]:
# Game Loop
while running :
    for event in pygame.event.get() :
        if event.type == pygame.QUIT :
                running = False
    current_time = pygame.time.get_ticks()
    if current_time - last_pipe > 2000 and Active :
        random_height = random.randint(200,576)
        topPipe = Pipe(1044,random_height,False)
        downPipe = Pipe(1044,random_height,True)
        pipe_group.add(topPipe)
        pipe_group.add(downPipe)
        last_pipe = current_time

    if len(pipe_group) > 0:
        if bird_group.sprites()[0].rect.left > pipe_group.sprites()[0].rect.left\
        and bird_group.sprites()[0].rect.right < pipe_group.sprites()[0].rect.right\
        and pass_pipe == False:
            pass_pipe = True
    if pass_pipe == True and len(pipe_group) > 0 :
        if bird_group.sprites()[0].rect.left > pipe_group.sprites()[0].rect.right:
            score += 1
            pass_pipe = False
    screen.fill(BACKGROUND_COLOR)
    text = font.render(f'Score : {str(score)}',True,(0,0,0))
    bird.draw()
    bird.jump()
    pipe_group.draw(screen)
    Active = check_collision(bird_group,pipe_group)
    if Active :
        bird_group.update()
        pipe_group.update()
    screen.blit(text,(400,30))
    pygame.display.update()
    clock.tick(60) 
    if not Active :
        if 1 in pygame.key.get_pressed() :
            bird_group.sprites()[0].rect.center = (300,250)
            bird_group.sprites()[0].vel = 0
            score = 0
            pipe_group.empty()
            Active = True